# Agentic Travel Planner

In [ ]:
pip install -U langchain langchain-core langchain-community langchain-openai langchain-groq

In [3]:
!pip install --quiet langchain openai requests python-dotenv pydantic amadeus langchain_openai langchain_groq
!pip install --quiet langchain openai requests python-dotenv pydantic amadeus langchain_openai langchain_groq gradio

 Imports & helper functions

In [39]:
from langchain_openai import ChatOpenAI
from langchain_classic.agents import create_react_agent
from langchain_classic.agents.agent import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from typing import List

using open ai key and making it secure

In [5]:
from google.colab import userdata
key=userdata.get('groq_apikey')

In [6]:
groq_key = userdata.get('groq_apikey')

from groq import Groq
client = Groq(api_key=groq_key)

completion = client.chat.completions.create(
    model="openai/gpt-oss-20b",
    messages=[{"role": "user", "content": "Hello, how are you?"}]
)

print(completion.choices[0].message.content)


Hello! I'm doing great—thanks for asking. How are you doing today?


In [10]:
from langchain.agents import create_agent


importing weather api

In [11]:
weather_key = userdata.get('rapidapi-key')

import http.client

conn = http.client.HTTPSConnection("open-weather13.p.rapidapi.com")

headers = {
    "x-rapidapi-key": weather_key,
    "x-rapidapi-host": "open-weather13.p.rapidapi.com"
}

conn.request("GET", "/city?city=Lahore&lang=EN", headers=headers)
res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))


{"coord":{"lon":74.3436,"lat":31.5497},"weather":[{"id":711,"main":"Smoke","description":"smoke","icon":"50n"}],"base":"stations","main":{"temp":57.18,"feels_like":55.74,"temp_min":57.18,"temp_max":57.18,"pressure":1018,"humidity":67,"sea_level":1018,"grnd_level":993},"visibility":4000,"wind":{"speed":0,"deg":0},"clouds":{"all":100},"dt":1765308026,"sys":{"type":1,"id":7585,"country":"PK","sunrise":1765331476,"sunset":1765367983},"timezone":18000,"id":1172451,"name":"Lahore","cod":200}


In [12]:
@tool
def get_weather(city: str) -> dict:
    """Fetch weather for a city using RapidAPI."""
    weather_key = userdata.get('rapidapi-key')

    url = "https://open-weather13.p.rapidapi.com/city"
    params = {"city": city, "lang": "EN"}

    headers = {
        "x-rapidapi-key": weather_key,
        "x-rapidapi-host": "open-weather13.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status() # Raise an exception for HTTP errors
    return response.json()



agent with weather tool only

In [13]:
groq_api_key = userdata.get('groq_apikey')

model = ChatGroq(
    temperature=0,
    model="openai/gpt-oss-20b",
    groq_api_key=groq_api_key
)

agent = create_agent(
    model=model,
    tools=[get_weather],
    system_prompt="You are a helpful assistant."
)

city = input("Enter your city: ")

response = agent.invoke({
    "messages": [{"role": "user", "content": f"What is the weather in {city}?"}]
})

print(response["messages"][-1].content)


Enter your city: peshawar
**Peshawar – Current Weather**

| Item | Value |
|------|-------|
| **Temperature** | 10 °C (≈ 50 °F) |
| **Feels Like** | 8 °C (≈ 47 °F) |
| **Weather** | Haze – light haze in the air |
| **Humidity** | 57 % |
| **Wind** | Calm (0 m/s) |
| **Visibility** | 4 km |
| **Cloud Cover** | 20 % |
| **Pressure** | 1019 hPa |

**Quick Takeaway**

- It’s a mild day with a light haze, so visibility is a bit reduced.
- The air feels slightly cooler than the actual temperature (feels‑like 8 °C).
- Wind is negligible, so it’s a calm day overall.

Enjoy your day in Peshawar! If you need more details (e.g., sunrise/sunset times or a forecast), just let me know.


importing hotel api

In [26]:
from google.colab import userdata
key=userdata.get('hotelapii')

"https://tripadvisor16.p.rapidapi.com/api/v1/hotels/searchLocation"

In [15]:
hotel_key = userdata.get('hotelapii')

url = "https://tripadvisor16.p.rapidapi.com/api/v1/hotels/searchLocation"
params = {"query": "Lahore"}

headers = {
    "x-rapidapi-key": hotel_key,
    "x-rapidapi-host": "tripadvisor16.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=params)
print(response.json())


{'message': 'You have exceeded the MONTHLY quota for Requests on your current plan, BASIC. Upgrade your plan at https://rapidapi.com/DataCrawler/api/tripadvisor16'}


In [16]:
@tool
def get_hotel(city: str) -> dict:
    """Fetch hotel data for a city."""
    hotel_key = userdata.get('hotelapii')

    url = "https://tripadvisor16.p.rapidapi.com/api/v1/hotels/searchLocation"
    params = {"query": city}

    headers = {
        "x-rapidapi-key": hotel_key,
        "x-rapidapi-host": "tripadvisor16.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status() # Raise an exception for HTTP errors
    return response.json()

In [17]:

@tool
def get_places(city: str) -> dict:
    """Fetch popular attractions and places for a city."""
    places_key = userdata.get('placesapi')   # <---- Your API stored here

    url = "https://api.foursquare.com/v3/places/search"
    params = {
        "query": "attractions",
        "near": city,
        "limit": 10
    }

    headers = {
        "Accept": "application/json",
        "Authorization": places_key
    }

    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

agents with weather tool and hotel tool

In [28]:
agent = create_agent(
    model=model,
    tools=[get_weather, get_hotel],
    system_prompt="You are a helpful assistant."
)

city = input("Enter city: ")

response = agent.invoke({
    "messages": [{"role": "user", "content": f"Give hotel prices, weather conditions and suggestions for {city}"}]
})

print(response["messages"][-1].content)


Enter city: islamabad
**Islamabad – Quick Snapshot**

| Item | Details |
|------|---------|
| **Weather (now)** | Clear skies, 12 °C (≈53 °F). Light breeze (≈4 m/s). Good for outdoor activities. |
| **Typical Hotel Prices** | • **Budget**: PKR 3,000–6,000 ($15–$30) per night (e.g., Hotel 1, Hotel 2) <br>• **Mid‑range**: PKR 7,000–15,000 ($35–$75) per night (e.g., Iqbal Hotel, Centaurus Hotel) <br>• **Luxury**: PKR 16,000–35,000 ($80–$170) per night (e.g., Serena Hotel, Marriott, Islamabad Marriott) |
| **Top Attractions** | • Faisal Mosque (iconic mosque) <br>• Daman‑e‑Koh (hilltop viewpoint) <br>• Pakistan Monument & Museum <br>• Rawal Lake & Safari Park <br>• Lok Virsa Museum (cultural heritage) <br>• Blue Area & Centaurus Mall (shopping & dining) |
| **Day‑Trip Ideas** | • Murree (hill station, ~1.5 h drive) <br>• Taxila (ancient Buddhist ruins, ~1 h drive) <br>• Margalla Hills National Park (hiking, wildlife) |
| **Food & Drink** | • Try local dishes: *Chapli Kebab*, *Nihari*, *Sam

getiing flights api

In [19]:
from google.colab import userdata
key=userdata.get('flightsapi_rapid')

"https://kiwi-com-cheap-flights.p.rapidapi.com/round-trip"

In [20]:
import requests

url = "https://kiwi-com-cheap-flights.p.rapidapi.com/round-trip"

querystring = {"source":"Country:PK,Country:US,Country:UK","destination":"City:Lahore_PK,City:WashingtonDC_US,City:London_UK","currency":"usd","locale":"en","adults":"1","children":"0","infants":"0","handbags":"1","holdbags":"0","cabinClass":"ECONOMY","sortBy":"QUALITY","sortOrder":"ASCENDING","applyMixedClasses":"true","allowReturnFromDifferentCity":"true","allowChangeInboundDestination":"true","allowChangeInboundSource":"true","allowDifferentStationConnection":"true","enableSelfTransfer":"true","allowOvernightStopover":"true","enableTrueHiddenCity":"true","enableThrowAwayTicketing":"true","outbound":"SUNDAY,WEDNESDAY,THURSDAY,FRIDAY,SATURDAY,MONDAY,TUESDAY","transportTypes":"FLIGHT","contentProviders":"FLIXBUS_DIRECTS,FRESH,KAYAK,KIWI","limit":"20"}

headers = {
	"x-rapidapi-key": key,
	"x-rapidapi-host": "kiwi-com-cheap-flights.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'__typename': 'Itineraries', 'metadata': {'topFiveResultsBaggageEligibility': None, 'carriers': [], 'stopoverCountries': [], 'inboundDays': [], 'outboundDays': [], 'travelTips': [], 'topResults': {'best': None, 'cheapest': None, 'fastest': None, 'sourceTakeoffAsc': None, 'destinationLandingAsc': None}, 'priceAlertExists': None, 'itinerariesCount': 0, 'hasMorePending': False, 'missingProviders': [], 'statusPerProvider': [{'provider': {'id': 'ContentProvider:KIWI'}, 'errorHappened': True, 'errorMessage': 'downstream error, code:422'}], 'topFiveOriginalItinerariesContainTHC': False, 'hasTier1MarketItineraries': False, 'contextuallyRecommendedFilters': [{'__typename': 'AirportChangesInfo', 'count': 0, 'ranks': []}, {'__typename': 'DirectFlightsInfo', 'count': 0, 'ranks': []}, {'__typename': 'MultipleStopsInfo', 'count': 0, 'ranks': []}, {'__typename': 'SelfTransferInfo', 'count': 0, 'ranks': []}, {'__typename': 'HiddenCityInfo', 'count': 0, 'ranks': []}, {'__typename': 'WeekendTripInfo', 

In [21]:
@tool
def get_flights(destination_city: str, source_country_code="PK", adults: int = 1) -> dict:
    """Fetch flight data using Kiwi API."""

    flights_key = userdata.get('flightsapi_rapid')

    url = "https://kiwi-com-cheap-flights.p.rapidapi.com/round-trip"

    params = {
        "source": f"Country:{source_country_code}", # Completed the f-string
        "destination": f"City:{destination_city}",
        "currency": "usd",
        "locale": "en",
        "adults": str(adults),
        "limit": "20"
    }

    headers = {
        "x-rapidapi-key": flights_key,
        "x-rapidapi-host": "kiwi-com-cheap-flights.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status() # Raise an exception for HTTP errors
    return response.json()

In [22]:
agent = create_agent(
    model=model,
    tools=[get_flights],
    system_prompt="You are a helpful assistant."
)

city = input("Enter city: ")

response = agent.invoke({
    "messages": [{"role": "user", "content": f"Give flight prices and suggestions for {city}"}]
})

print(response["messages"][-1].content)

Enter city: peshawar
I’m sorry, but the Kiwi API didn’t return any flight data for a trip to Peshawar at the moment (the request resulted in a downstream error). I can still help you plan your trip by giving you some general guidance on flight prices, airlines, and travel tips for Peshawar. If you’d like, you can also try searching on other flight‑search engines (Google Flights, Skyscanner, Kayak, or the airlines’ own sites) for the most up‑to‑date prices.

---

## 1. Typical Flight Prices (2024‑2025)

| From | To | Typical One‑Way Price (USD) | Notes |
|------|----|-----------------------------|-------|
| Islamabad (ISB) | Peshawar (PEW) | $30–$70 | Direct flights are usually the cheapest; check for promotions. |
| Karachi (KHI) | Peshawar (PEW) | $40–$80 | Direct flights available; sometimes cheaper if you book early. |
| Lahore (LHE) | Peshawar (PEW) | $35–$75 | Direct flights; Lahore is a major hub so you’ll find frequent services. |
| International (e.g., Dubai, Doha, Istanbul) | 

In [23]:

@tool
def estimate_budget(avg_hotel_price: float, avg_flight_price: float, days: int = 3) -> str:
    """Simple travel cost estimation."""

    total = (avg_hotel_price * days) + avg_flight_price

    return (
        f"Estimated cost breakdown:\n"
        f"- Flight cost: ${avg_flight_price}\n"
        f"- Hotel cost for {days} days: ${avg_hotel_price * days}\n"
        f"- Total Estimated Trip Cost: ${total}"
    )

**Final agent with weather +hotels+flights**

In [32]:
agent = create_agent(
    model=model,
    tools=[get_weather, get_hotel, get_flights,estimate_budget],
system_prompt = (
    "You are a friendly travel planner. "
        "Your goal is to create a comprehensive 3-day itinerary. "
        "Always call the tools to gather flights, hotel data, weather, and popular attractions (POIs). "
        "Use the estimate_budget tool to provide a simple cost breakdown if the user provides average flight and hotel prices. "
        "When you have gathered all necessary information, reply ONLY with a well-structured, day-by-day itinerary and the budget estimate."

))

city = input("Enter destination city: ")

response = agent.invoke({
    "messages": [{"role": "user", "content": f"Give Give flight prices and suggestions , hotel suggestions, and weather for {city}"}]
})

print(response["messages"][-1].content)


Enter destination city: bradfore
**Bradford – Quick Travel Snapshot**

| Category | Details |
|----------|---------|
| **Flights** | • No direct Kiwi itineraries returned for “Bradford” (likely because the city is served by the nearby **Leeds Bradford Airport – LBA**). <br>• Typical one‑way fares from major hubs (e.g., London, New York, Dubai) range **$70–$250** depending on season and airline. <br>• **Alternative airports**: <br>  – **Manchester Airport (MAN)** – ~30 min drive, more flight options. <br>  – **Leeds Bradford Airport (LBA)** – local flights, often cheaper. |
| **Hotels** | • **The Grand Hotel, Bradford** – 4‑star, central location, rooms $120–$180/night. <br>• **The Old Mill Hotel** – boutique, historic charm, rooms $90–$140/night. <br>• **Premier Inn Bradford City Centre** – budget, rooms $60–$90/night. <br>• **Holiday Inn Express Bradford** – modern, rooms $110–$160/night. <br>• **Best‑value**: Book early or use discount codes; check for free cancellation. |
| **Weathe

creating a gradio interface

In [34]:
from io import StringIO
# import gradio as gr and from langchain_core.messages import HumanMessage are already there


In [36]:
agent = create_agent(
    model=model,
    tools=[get_weather, get_hotel, get_flights, estimate_budget, get_places],
    system_prompt=(
        "You are a friendly travel planner. "
        "Always call the tools to gather flights, weather, hotels, attractions, and estimate the budget. "
        "When you have everything, reply ONLY with a 3-day itinerary including cost breakdown."
    )
)


In [38]:
import gradio as gr
from langchain_core.messages import HumanMessage

def run_agent_in_gradio(destination: str, start_date: str, end_date: str) -> str:
    if not destination or not start_date or not end_date:
        return "Please enter all fields: destination, start date, and end date."

    # Construct a structured prompt for the agent
    user_prompt = (
        f"Plan a trip to {destination} from {start_date} to {end_date}.\n"
        "Include:\n"
        "1. Day-by-day itinerary for the duration of the trip\n"
        "2. Weather forecast for each day\n"
        "3. Hotel suggestions with approximate costs\n"
        "4. Flight options with approximate costs\n"
        "5. Total estimated budget\n"
        "Use the tools (get_weather, get_hotel, get_flights, estimate_budget) as needed."
    )

    try:
        response = agent.invoke({
            "messages": [HumanMessage(content=user_prompt)]
        }, verbose=True)

        logs = "=== TOOL CALLS & INTERMEDIATE STEPS ===\n"

        if "intermediate_steps" in response:
            for step in response["intermediate_steps"]:
                tool_name = step[0].name if hasattr(step[0], 'name') else str(step[0])
                tool_output = step[1]
                logs += f"Tool: {tool_name}\nOutput:\n{tool_output}\n\n"
        else:
            logs += "Tool call details not available. Ensure agent is verbose=True.\n\n"

        final = response["messages"][-1].content if "messages" in response else str(response)
        return logs + "=== FINAL ITINERARY ===\n" + final

    except Exception as e:
        return f"An error occurred: {e}"

# Gradio interface with structured inputs
iface = gr.Interface(
    fn=run_agent_in_gradio,
    inputs=[
        gr.Textbox(label="Destination City", placeholder="E.g., London"),
        gr.Textbox(label="Start Date", placeholder="YYYY-MM-DD"),
        gr.Textbox(label="End Date", placeholder="YYYY-MM-DD")
    ],
    outputs=gr.Markdown(label="Travel Itinerary"),
    title="Agentic Travel Planner",
    description="Enter destination, start date, and end date. The agent will plan your trip with flights, hotels, weather, and cost estimates."
)

iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://638d71efafec62fa40.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
